### Mount gdrive

### This cell is for Google colaboratory users.

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
%cd /gdrive/My\ Drive/DNNTopology

### Parameter setting

In [0]:
class_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
firstLayerSize = 512
secondLayerSize = 512
outputNeuron = 10

outputDir = "./models/"
epochs = 30
batch_size = 32
model_name = outputDir + "firstLayerSize"+str(firstLayerSize)+"secondLayerSize"+str(secondLayerSize)+"outputNeuron"+str(outputNeuron)+ "class" + str(class_list) + ".h5"

### Import libraries

In [0]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np
%matplotlib inline

### Load CIFAR10 data set

In [15]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
# Extract data in class_list
train_filter = np.where( [ (element in class_list) for element in y_train] )
test_filter = np.where( [ (element in class_list) for element in y_test] )
x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

In [0]:
# Normalize image data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
# Convert vectors to binary class
y_train = keras.utils.to_categorical(y_train, outputNeuron)
y_test = keras.utils.to_categorical(y_test, outputNeuron)

### Make model

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(firstLayerSize, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(secondLayerSize, activation = 'relu'))
model.add(Dense(outputNeuron, activation = 'softmax'))

In [0]:
model.summary()

### Train model

In [0]:
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
fit = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

In [22]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 1s 71us/step
Test loss: 0.6729544461250305
Test accuracy: 0.7753


### Save model

In [0]:
model.save(model_name, include_optimizer = False)